Evaluate google translate

In [6]:
from pathlib import Path
import sys
root = Path.cwd().parent
sys.path.append(str(root))

from google.cloud import translate_v2 as gg_translate
import pandas as pd
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/mac/Downloads/friendly-drake-365016-5060bb0f2dff.json"

In [7]:
sentences = pd.read_csv('CWQ Annotation - EN.csv')
sentences = list(sentences['questionWithBrackets'].dropna())

In [8]:
translate_client = gg_translate.Client()
def translate_with_qmark(q, target):
    q = q if q[-1] == "?" else q + "?"
    q_trans = translate_client.translate(q, target_language=target)["translatedText"]
    return q_trans[:-1] if q_trans[-1] == "?" else q_trans

In [10]:
jp = []
for en in sentences:
    jp.append(translate_with_qmark(en, 'ja'))

In [12]:
dic = [{'questionWithBrackets':en, 'questionWithBrackets_jp_gg':jp} for en,jp in zip(sentences, jp)]

In [13]:
# write a file for manual assessment
import csv
with open('CWQ Annotation - JP - Google.csv','w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=['questionWithBrackets', 'questionWithBrackets_jp_gg'])
    writer.writeheader()
    writer.writerows(dic)

In [16]:
# reference-based assessment
from sacrebleu.metrics import BLEU
from sacrebleu.tokenizers import tokenizer_ja_mecab

In [18]:
sentences = pd.read_csv('CWQ Annotation - GOLD-JP.csv')
sentences_en = list(sentences['questionPatternModEntities'].dropna())
gold_jp = list(sentences['questionPatternModEntitiesJP'].dropna())

In [19]:
infer_jp = []
for en in sentences_en:
    infer_jp.append(translate_with_qmark(en, 'ja'))

In [20]:
bleu = BLEU(tokenize='ja-mecab')
bleu.corpus_score(hypotheses=infer_jp, references=[gold_jp])

BLEU = 42.36 76.9/53.7/38.4/26.8 (BP = 0.933 ratio = 0.935 hyp_len = 3266 ref_len = 3494)

BLEU = 42.36 76.9/53.7/38.4/26.8 (BP = 0.933 ratio = 0.935 hyp_len = 3266 ref_len = 3494)

In [21]:
infer_jp

['M0は脚本家だった？',
 '作曲家はM0だった？',
 'M0 は何を見つけましたか',
 'M0 は何を編集しましたか',
 'M0 M1の続編だった？',
 'M1の続編はM0だった？',
 'M0 の親は',
 'M0は何を書いたの？',
 'M0 は M1 に雇用されていましたか',
 'M0はM1によって書かれましたか',
 'M0の幹部はM1がプロデュース？',
 'M1の衣装デザイナーは作曲家だった？',
 'M1の撮影監督は脚本家だった？',
 'M1 の配偶者は M0 の従業員でしたか',
 'M1は作曲家によって制作されましたか',
 'M1の作曲者は脚本家？',
 'M1の作曲者は撮影監督ですか？',
 'M0が配信したM1の続編は',
 'M0が主演したM1の続編は',
 'M0 の創業者は M1 の親でしたか',
 'M0 の兄弟は M1 の従業員でしたか',
 'M0 のどの雇用主が M1 のディストリビューターでしたか',
 'M0 の親は M1 のどの雇用主でしたか',
 '衣装デザイナーの配偶者M0は兄弟だった？',
 '衣装デザイナーの配偶者がM0の創業者だった？',
 'M0の脚本・監督作品は？',
 'M0 が何を書き、何を編集したか',
 '衣装デザイナーM2の両親はスウェーデン人だった？',
 'M2の配偶者は日本人の撮影監督だった？',
 'M0 が見つけた映画監督の雇用主は',
 'M0 によって書かれ、制作されたものは何ですか',
 '衣装デザイナーの雇用主はM0のプロデューサーだった？',
 '衣装デザイナーの兄弟はM0の配偶者だった？',
 'M0 のどの雇用主が会社の親でしたか',
 '映画の配給会社だったM0の雇用主は',
 'M1 のプロデューサーとディストリビューターは M0 によって設立されましたか',
 '衣装デザイナーはM2のスウェーデン人の配偶者でしたか',
 'M1と結婚したM0の配偶者と兄弟は',
 'M0 が見つけた M1 のディストリビューターとプロデューサーは',
 'M1 の創設者は M0 のオランダ人の兄弟でしたか',
 'M0 M1 の従業員のドイツ人の兄弟でしたか',
 'M0によって設立されたM1のディストリビューターおよびプロデューサーは',
 'M0 が書き、M1 が

In [34]:
def rep(sen):
    sen = sen.replace('だった','')
    sen = sen.replace('？','')
    sen = sen.replace('でしたか','')
    sen = sen.replace('ですか','')
    sen = sen.replace('によって','に')
    sen = sen.replace(' ','')
    return sen
new_infer_jp = []
new_gold_jp = []
for inf, gol in zip(infer_jp, gold_jp):
    new_infer_jp.append(rep(inf))
    new_gold_jp.append(rep(gol))



In [35]:
bleu = BLEU(tokenize='ja-mecab')
bleu.corpus_score(hypotheses=new_infer_jp, references=[new_gold_jp])

BLEU = 45.07 79.0/56.7/41.4/29.3 (BP = 0.934 ratio = 0.936 hyp_len = 3102 ref_len = 3315)

In [36]:
new_infer_jp

['M0は脚本家',
 '作曲家はM0',
 'M0は何を見つけましたか',
 'M0は何を編集しましたか',
 'M0M1の続編',
 'M1の続編はM0',
 'M0の親は',
 'M0は何を書いたの',
 'M0はM1に雇用されていましたか',
 'M0はM1に書かれましたか',
 'M0の幹部はM1がプロデュース',
 'M1の衣装デザイナーは作曲家',
 'M1の撮影監督は脚本家',
 'M1の配偶者はM0の従業員',
 'M1は作曲家に制作されましたか',
 'M1の作曲者は脚本家',
 'M1の作曲者は撮影監督',
 'M0が配信したM1の続編は',
 'M0が主演したM1の続編は',
 'M0の創業者はM1の親',
 'M0の兄弟はM1の従業員',
 'M0のどの雇用主がM1のディストリビューター',
 'M0の親はM1のどの雇用主',
 '衣装デザイナーの配偶者M0は兄弟',
 '衣装デザイナーの配偶者がM0の創業者',
 'M0の脚本・監督作品は',
 'M0が何を書き、何を編集したか',
 '衣装デザイナーM2の両親はスウェーデン人',
 'M2の配偶者は日本人の撮影監督',
 'M0が見つけた映画監督の雇用主は',
 'M0に書かれ、制作されたものは何',
 '衣装デザイナーの雇用主はM0のプロデューサー',
 '衣装デザイナーの兄弟はM0の配偶者',
 'M0のどの雇用主が会社の親',
 '映画の配給会社M0の雇用主は',
 'M1のプロデューサーとディストリビューターはM0に設立されましたか',
 '衣装デザイナーはM2のスウェーデン人の配偶者',
 'M1と結婚したM0の配偶者と兄弟は',
 'M0が見つけたM1のディストリビューターとプロデューサーは',
 'M1の創設者はM0のオランダ人の兄弟',
 'M0M1の従業員のドイツ人の兄弟',
 'M0に設立されたM1のディストリビューターおよびプロデューサーは',
 'M0が書き、M1がプロデュースしたのは',
 'ある会社の創業者M0はフランス人の兄弟',
 'M0はM2に作成され、M1に作成されましたか',
 'M0が制作し、M1が執筆したエグゼクティブは何',
 'M0に設立されたM1のディストリビューターおよびプロデューサーは',
 'M0の親、配偶者、兄弟はM1と結婚しました

In [33]:
new_gold_jp

['M0は脚本家',
 'ある作曲家はM0',
 'M0が創設したのは何',
 'M0が編集したのは何',
 'M0はM1の続編',
 'M1の続編はM0',
 'M0の親は何',
 '何がM0に書かれましたか',
 'M0はM1に雇用されましたか',
 'M0はM1に書かれましたか',
 'M0はM1にエグゼクティブプロデュースされましたか',
 'M1の衣裳デザイナーは作曲家',
 'M1の撮影監督は脚本家',
 'M1の配偶者はM0の被雇用者',
 'M1は作曲家にプロデュースされましたか',
 'M1のある作曲家は脚本家',
 'M1のある作曲家は撮影監督',
 'M0が配給したのはM1のどの続編',
 'M1のどの続編はM0を主演させましたか',
 'M0の創設者はM1の親',
 'M0の兄弟姉妹はM1の被雇用者',
 'M1の配給者はM0のどの雇用主',
 'M1のどの雇用主はM0の親会社',
 'ある衣裳デザイナーの配偶者はM0の兄弟姉妹',
 'ある衣裳デザイナーの配偶者はM0の創設者',
 '何がM0に書かれ、監督されましたか',
 '何がM0に書かれ、編集されましたか',
 'ある衣裳デザイナーはM2のスウェーデン人の親',
 'M2の配偶者は日本の撮影監督',
 'M0が創設したのはどの映画監督の雇用主',
 '何がM0に書かれ、プロデュースされましたか',
 'ある衣裳デザイナーの雇用主はM0のプロデューサー',
 'ある衣裳デザイナーの兄弟姉妹はM0の配偶者',
 'ある会社の親会社はM0のどの雇用主',
 'ある映画の配給者はM0のどの雇用主',
 'M1のプロデューサーと配給者はM0に創設されましたか',
 'ある衣裳デザイナーはM2のスウェーデン人の配偶者',
 'M0のどの配偶者と兄弟姉妹がM1と結婚しましたか',
 'M0はM1のどの配給者とプロデューサーを創設しましたか',
 'M1の創設者はM0のオランダ人の兄弟姉妹',
 'M0のあるドイツ人の兄弟姉妹はM1の被雇用者',
 'M1のどの配給者とプロデューサーがM0に創設されましたか',
 '何がM0に書かれ、M1にプロデュースされましたか',
 'ある会社の創設者はM0のフランス人の兄弟姉妹',
 'M0はM2にプロデュースされ、M1に書かれましたか',
 '何がM0にエグ